In [1]:
from livecellx.annotation.labelme2coco import get_coco_from_labelme_folder

labelme_folder = r"D:\LiveCellTracker-dev\datasets\cxa-high-density\2023-12-4-fixed_CXA\0.25^4_wellF6_RI_MIP_stitched"
dataset_folder_path = r"D:\LiveCellTracker-dev\datasets\cxa-high-density\contrast_enhanced_factor=1.5_png_data\0.25^4_wellF6_RI_MIP_stitched"
coco = get_coco_from_labelme_folder(labelme_folder, dataset_folder_path=dataset_folder_path, image_file_ext="png")
coco

In [ ]:
type(coco), coco.getImgIds()

In [ ]:
from livecellx.annotation.coco_utils import coco_to_scs

scs = coco_to_scs(coco)

In [ ]:
scs[0]

In [ ]:
scs[0].meta

In [ ]:
dataset_folder_path

In [ ]:
from livecellx.core.datasets import LiveCellImageDataset
from pathlib import Path

def _get_time_from_filepath(filename: str):
    """{parent_dirs}/{time}.png"""
    return int(Path(filename).name.split(".")[0])

time2url = {int(_get_time_from_filepath(p)): p for p in Path(dataset_folder_path).glob("*.png")}
all_img_dataset = LiveCellImageDataset(time2url=time2url)

In [ ]:
time2url

In [ ]:
set([sc.timeframe for sc in scs])

In [ ]:
idx = 32
scs[idx].img_dataset.time2url, scs[idx].timeframe, scs[idx].show_panel(padding=100)

In [ ]:
all_img_dataset

In [ ]:
from livecellx.core.sct_operator import create_sctc_edit_viewer_by_interval, create_sctc_from_scs

def reindex_scs_times(scs):
    """Reindex scs timeframes to 0, 1, 2, ..."""
    scs = [sc.copy() for sc in scs]
    all_times = sorted(set([sc.timeframe for sc in scs]))
    time2newtime = {t: i for i, t in enumerate(all_times)}
    for sc in scs:
        sc.timeframe = time2newtime[sc.timeframe]
    return scs
# sctc = create_sctc_from_scs(reindex_scs_times(scs))
sctc = create_sctc_from_scs(scs)
sct_operator = create_sctc_edit_viewer_by_interval(sctc, img_dataset=all_img_dataset, span_interval=200)